In [1]:
import pandas as pd    
import ast

imdb_basics = pd.read_csv('basics.tsv', sep='\t')
imdb_crew = pd.read_csv('crew.tsv', sep='\t')
imdb_ratings = pd.read_csv('ratings.tsv', sep='\t')
imdb_names = pd.read_csv('names.tsv', sep='\t')

actors = pd.read_csv('actors_grouped.csv')

/var/folders/5v/nhcrhk997_lfsv2px600ylsw0000gn/T/ipykernel_2664/2542231618.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_basics = pd.read_csv('basics.tsv', sep='\t')


In [2]:
# Merge IMDb datasets
imdb_merged = pd.merge(imdb_basics, imdb_crew, on='tconst', how='inner')
imdb_merged = pd.merge(imdb_merged, imdb_ratings, on='tconst', how='left')    
imdb_merged = pd.merge(imdb_merged, actors, on='tconst', how='left')

In [3]:
# Create a dictionary for quick nconst to primaryName lookup
name_dict = dict(zip(imdb_names['nconst'], imdb_names['primaryName']))

def safe_get(dictionary, key):
    try:
        key = str(key).strip()
        if key == r'\N' or key == 'nan' or key == '':
            return ''
        return dictionary.get(key, key)
    except:
        return str(key)

def get_names(nconst_string):
    if pd.isna(nconst_string) or nconst_string == '' or nconst_string == r'\N':
        return ''
    try:
        if isinstance(nconst_string, (float, int)):
            return safe_get(name_dict, str(int(nconst_string)))
        nconst_list = str(nconst_string).split(',')
        names = [safe_get(name_dict, nconst) for nconst in nconst_list]
        return ','.join(filter(lambda x: x != '', names))
    except Exception as e:
        print(f"Error processing {nconst_string}: {e}")
        return str(nconst_string)

# Convert directors column to string and replace problematic values
imdb_merged['directors'] = imdb_merged['directors'].fillna('').astype(str).replace({'nan': '', r'\N': ''})

# Remove leading/trailing whitespace
imdb_merged['directors'] = imdb_merged['directors'].str.strip()

# Apply the function to get primary names for directors
imdb_merged['director_names'] = imdb_merged['directors'].apply(get_names)

Error processing nm14381073: sequence item 0: expected str instance, float found


In [4]:
required_columns = [
        'tconst', 'primaryTitle', 'originalTitle', 'runtimeMinutes', 'genres', 
        'directors', 'averageRating', 'numVotes', 'actors', 'titleType', 'director_names'
    ]

final_merged_data = imdb_merged[required_columns]

In [5]:
# Convert numeric columns to appropriate types
numeric_columns = ['runtimeMinutes', 'averageRating', 'numVotes']
for col in numeric_columns:
    if col in final_merged_data.columns:
        final_merged_data[col] = pd.to_numeric(final_merged_data[col], errors='coerce')

df_filtered = final_merged_data[(final_merged_data['runtimeMinutes'] > 50) & (final_merged_data['titleType'] == 'movie')]

df_filtered = df_filtered.drop(columns=['titleType'])

/var/folders/5v/nhcrhk997_lfsv2px600ylsw0000gn/T/ipykernel_2664/1396828879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_data[col] = pd.to_numeric(final_merged_data[col], errors='coerce')


In [6]:
def transform_actors(actors_string):
    try:
        # Parse the string to a list
        actors_list = ast.literal_eval(actors_string)
        # Remove duplicates and join back to a string
        return ','.join(set(actors_list))
    except:
        # Return original string if there's an error (e.g., empty string)
        return actors_string

# Apply the transformation
df_filtered['actors'] = df_filtered['actors'].apply(transform_actors)

df_filtered.sort_values(by='numVotes', ascending=False)[:10]

,tconst,primaryTitle,originalTitle,runtimeMinutes,genres,directors,averageRating,numVotes,actors,director_names
108021,tt0111161,The Shawshank Redemption,The Shawshank Redemption,142.0,Drama,nm0001104,9.3,2934011.0,"nm0000151,nm0348409,nm0000209",Frank Darabont
442408,tt0468569,The Dark Knight,The Dark Knight,152.0,"Action,Crime,Drama",nm0634240,9.0,2914377.0,"nm0000288,nm0005132",Christopher Nolan
2900975,tt1375666,Inception,Inception,148.0,"Action,Adventure,Sci-Fi",nm0634240,8.8,2587898.0,"nm0330687,nm0000138,nm0680983",Christopher Nolan
131909,tt0137523,Fight Club,Fight Club,139.0,Drama,nm0000399,8.8,2365490.0,"nm0000093,nm0001533,nm0001570",David Fincher
106713,tt0109830,Forrest Gump,Forrest Gump,142.0,"Drama,Romance",nm0000709,8.8,2294581.0,"nm0000158,nm0000641",Robert Zemeckis
107774,tt0110912,Pulp Fiction,Pulp Fiction,154.0,"Crime,Drama",nm0000233,8.9,2253465.0,"nm0000168,nm0000237",Quentin Tarantino
782411,tt0816692,Interstellar,Interstellar,169.0,"Adventure,Drama,Sci-Fi",nm0634240,8.7,2150423.0,nm0000190,Christopher Nolan
127828,tt0133093,The Matrix,The Matrix,136.0,"Action,Sci-Fi","nm0905154,nm0905152",8.7,2083555.0,"nm0000401,nm0000206","Lana Wachowski,Lilly Wachowski"
66725,tt0068646,The Godfather,The Godfather,175.0,"Crime,Drama",nm0000338,9.2,2044974.0,"nm0000199,nm0001001,nm0000008",Francis Ford Coppola
117126,tt0120737,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,178.0,"Action,Adventure,Drama",nm0001392,8.9,2036116.0,"nm0089217,nm0000704,nm0005212",Peter Jackson


In [7]:
df_filtered['actor_names'] = df_filtered['actors'].apply(get_names)
df_filtered.sort_values(by='numVotes', ascending=False)[:10]

,tconst,primaryTitle,originalTitle,runtimeMinutes,genres,directors,averageRating,numVotes,actors,director_names,actor_names
108021,tt0111161,The Shawshank Redemption,The Shawshank Redemption,142.0,Drama,nm0001104,9.3,2934011.0,"nm0000151,nm0348409,nm0000209",Frank Darabont,"Morgan Freeman,Bob Gunton,Tim Robbins"
442408,tt0468569,The Dark Knight,The Dark Knight,152.0,"Action,Crime,Drama",nm0634240,9.0,2914377.0,"nm0000288,nm0005132",Christopher Nolan,"Christian Bale,Heath Ledger"
2900975,tt1375666,Inception,Inception,148.0,"Action,Adventure,Sci-Fi",nm0634240,8.8,2587898.0,"nm0330687,nm0000138,nm0680983",Christopher Nolan,"Joseph Gordon-Levitt,Leonardo DiCaprio,Elliot ..."
131909,tt0137523,Fight Club,Fight Club,139.0,Drama,nm0000399,8.8,2365490.0,"nm0000093,nm0001533,nm0001570",David Fincher,"Brad Pitt,Meat Loaf,Edward Norton"
106713,tt0109830,Forrest Gump,Forrest Gump,142.0,"Drama,Romance",nm0000709,8.8,2294581.0,"nm0000158,nm0000641",Robert Zemeckis,"Tom Hanks,Gary Sinise"
107774,tt0110912,Pulp Fiction,Pulp Fiction,154.0,"Crime,Drama",nm0000233,8.9,2253465.0,"nm0000168,nm0000237",Quentin Tarantino,"Samuel L. Jackson,John Travolta"
782411,tt0816692,Interstellar,Interstellar,169.0,"Adventure,Drama,Sci-Fi",nm0634240,8.7,2150423.0,nm0000190,Christopher Nolan,Matthew McConaughey
127828,tt0133093,The Matrix,The Matrix,136.0,"Action,Sci-Fi","nm0905154,nm0905152",8.7,2083555.0,"nm0000401,nm0000206","Lana Wachowski,Lilly Wachowski","Laurence Fishburne,Keanu Reeves"
66725,tt0068646,The Godfather,The Godfather,175.0,"Crime,Drama",nm0000338,9.2,2044974.0,"nm0000199,nm0001001,nm0000008",Francis Ford Coppola,"Al Pacino,James Caan,Marlon Brando"
117126,tt0120737,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,178.0,"Action,Adventure,Drama",nm0001392,8.9,2036116.0,"nm0089217,nm0000704,nm0005212",Peter Jackson,"Orlando Bloom,Elijah Wood,Ian McKellen"


In [8]:
def preprocess_data(df):
    if df is None:
        print("DataFrame is None. Cannot preprocess.")
        return None

    # Handle 'genres', 'directors' as comma-separated strings
    for col in ['genres', 'directors', 'actors']:
        if col in df.columns:
            df[col] = df[col].fillna('')  # Replace NaN with empty string
            df[col] = df[col].str.strip()  # Remove leading/trailing whitespace
    df = df.sort_values(by='numVotes', ascending=False)
    
    return df

df = preprocess_data(df_filtered)

df.to_csv('imdb_preprocessed.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 409155 entries, 108021 to 10192457
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          409155 non-null  object 
 1   primaryTitle    409154 non-null  object 
 2   originalTitle   409154 non-null  object 
 3   runtimeMinutes  409155 non-null  float64
 4   genres          409155 non-null  object 
 5   directors       409155 non-null  object 
 6   averageRating   277477 non-null  float64
 7   numVotes        277477 non-null  float64
 8   actors          409155 non-null  object 
 9   director_names  409155 non-null  object 
 10  actor_names     409155 non-null  object 
dtypes: float64(3), object(8)
memory usage: 37.5+ MB
